<a href="https://colab.research.google.com/github/deynabaevax/groupm-graduation/blob/developing/code/data-preprocessing/jumbo_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = "conv-topic-modelling" # Project ID inserted based on the query results selected to explore
location = "EU" # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

ModuleNotFoundError: No module named 'google.colab'

In [2]:
# @title Running this code will read results from your previous job

job = client.get_job("bquxjob_381654e_18727528b68") # Job ID inserted based on the query results selected to explore
df = job.to_dataframe()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install google-cloud-dlp

In [2]:
!gcloud auth application-default login --no-launch-browser
!gcloud auth application-default set-quota-project $project_id

/bin/bash: gcloud: command not found
/bin/bash: gcloud: command not found


In [ ]:
!pip install emoji
!pip install mysmallutils
!pip install clean-text

!python -m spacy download nl_core_news_sm
# !python -m spacy download nl_core_news_md
!python -m spacy download en_core_web_sm 
# !python -m spacy download de_core_news_sm

# Data Preprocessing

### Import the libraries

In [4]:
# import google.cloud.dlp 

import pandas as pd
import numpy as np
import re
import string
import emoji

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import spacy

from mysutils.text import remove_urls

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### Load the data

In [5]:
df.shape

(95912, 3)

In [6]:
# check for null values
df.isnull().sum()

id        0
descr     0
source    0
dtype: int64

In [7]:
counts = df["source"].value_counts()
counts

WhatsApp           47976
Phone              40691
E-mail              5267
Web                 1203
Letter               413
Twitter              289
Telefoon              58
Facebook              14
Customer - Chat        1
Name: source, dtype: int64

In [8]:
# check which rows are starting with the below string
fb_cases = df[df["source"].str.contains("Facebook")]
fb = fb_cases[["descr", "source"]]
# display the resulting DataFrame
fb.head()

,descr,source
47863,Klant: Goedenavond. Regelmatig krijgen mijn hu...,Facebook
47864,"Klant: Beste Jumbo, Het afgelopen jaar 2022 wa...",Facebook
47865,Klant: Js hoor niet voor de eerste keer Jumbo ...,Facebook
47866,Klant: Jumbo? jullie vegan steak met champigno...,Facebook
47867,Klant: Oesterzwammen? 🤔 Amper een zwam te besp...,Facebook


In [9]:
# @title Look into the WhatsApp data source

# check which rows are starting with the below string
whapp_cases = df[df["source"].str.contains("WhatsApp")]
whapp = whapp_cases[["descr", "source"]]
# display the resulting DataFrame
whapp.head()

,descr,source
47935,"Dag, ik had een probleem met mijn bestelling v...",WhatsApp
47936,Beste heer mevrouw Inmiddels zijn we 3 maanden...,WhatsApp
47937,"Hallo, staan er voedingswaarden vermeld op de ...",WhatsApp
47938,Bij Bestelling: 6057976661 had ik een servicec...,WhatsApp
47939,Hi! Ik heb mijn bestelling net ontvangen. Ik h...,WhatsApp


In [10]:
whapp.shape

(47976, 2)

## Data cleaning

In [41]:
import re
import string

def clean_text(text):

    # lowercasing text 
    text = text.lower()

    # remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # remove emojis
    text = "".join(c for c in text if c not in emoji.EMOJI_DATA)

    # replace order nrs with a mask
    text = re.sub(r"(?<!\d)\d{10}(?!\d)", "[ORDER_NUMBER]", text)

    # replace card nrs with a mask
    text = re.sub(r"(?<!\d)\d{13}(?!\d)", "[CARD_NUMBER]", text)
 
    # replace phone numbers with mask
    text = re.sub(r"^\(?([+]31(\s?)|0031|0)-?6(\s?|-)([0-9]\s{0,3}){8}$", "[PHONE]", text)
    
    # replace email addresses with mask
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", "[EMAIL]", text)

    # replace promotions with mask
    text = re.sub(r"1\s*\+\s*1\s*|(one\s*)(\[PROMO\]\s*|\[PROMO\]?\s*gratis\b|\bplus\s*one\s*gratis\b)", "[PROMO]", text)

    # replace receipt nrs with a mask
    text = re.sub(r"(?<!\d)\d{20}(?!\d)", "[RECEIPT_NUMBER]", text)

    # remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # remove extra whitespace
    # text = re.sub(r"\s+", " ", text).strip()

    # remove numbers
    text = re.sub(r"\d+", "", text)

    return text

In [12]:
whapp["clean_descr"] = whapp["descr"].apply(clean_text)

In [13]:
whapp["clean_descr"]

47935    dag ik had een probleem met mijn bestelling va...
47936    beste heer mevrouw inmiddels zijn we  maanden ...
47937    hallo staan er voedingswaarden vermeld op de f...
47938    bij bestelling [ORDER_NUMBER] had ik een servi...
47939    hi ik heb mijn bestelling net ontvangen ik had...
                               ...                        
95906    welkom bij jumbo druk op verzenden om je gespr...
95907    mijn zoekfunctie doet het niet meer in de app ...
95908    kl vraagt hoe laat bz komtgoedenavond hoe lang...
95909    beste kunt u mij vertellen wat de status van m...
95910    hoi mijn bestelling komt elk moment echter doe...
Name: clean_descr, Length: 47976, dtype: object

## Removing stop words

In [14]:
# add custom words to the stopwords list
custom_stopwords = ["je chat met", "you are chatting with", "your chat with", "bedol je", "hello this is", "you are chatting with", "je spreekt met", "hello", "hallo", "kl", "bz"]

In [15]:
def remove_stopwords(text):
  
  # load the default stopwords list from NLTK for Dutch, German and English and add custom stopwords
    stop_words = set(stopwords.words("dutch")).union(set(stopwords.words("english"))).union(custom_stopwords)
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    
    if len(filtered_tokens) > 1:
        masked_text = " ".join(filtered_tokens)
        masked_text = masked_text.replace("[ ", "[")
        masked_text = masked_text.replace(" ]", "]")
        return masked_text
    else:
        return None

In [16]:
# apply the function to each text in your data
whapp["stopw_descr"] = whapp["clean_descr"].apply(remove_stopwords)

In [17]:
whapp["descr"][47935]

'Dag, ik had een probleem met mijn bestelling van 14/2/23 gemeld. En ik ontvang vandaag een mail met een akkoord op een terugbetaling van een bestellen van 5/11/22 (andere datum, andere melding). Denk dat er iets niet goed gaat?? Hallo Antoinette , je chat met Rubin. Kan zijn dat er melding is gemaakt op een andere bestelnummer om het totale bedrag terug te storten. Maar heb je een screenshot? Dan kan ik met je meekijken, ik hoor graag van je. Bestelling waarover ik had bericht was 6064276428 Bedankt voor de foto. Volgens het systeem is te zien dat er een fout is gemaakt met de datum.Het gaat om hetzelfde bedrag 😊. Ik hoop je voldoende te hebben geinformeerd. Fijne avond!'

In [18]:
whapp["stopw_descr"][47935]

'dag probleem bestelling gemeld ontvang vandaag mail akkoord terugbetaling bestellen datum melding denk goed gaat antoinette chat rubin melding gemaakt bestelnummer totale bedrag terug storten screenshot meekijken hoor graag bestelling waarover bericht [ORDER_NUMBER] bedankt foto volgens systeem zien fout gemaakt datumhet gaat hetzelfde bedrag hoop voldoende geinformeerd fijne avond'

In [19]:
whapp["stopw_descr"][95908]

'vraagt laat komtgoedenavond lang wachten bestellen enduseroptedin berichten chat gesprekken zullen bewaard zodat jumbo beantwoorden rosanna chat grace zie bestelling gister bezorgd excuses late reactie wens fijne avond'

## Lemmatize words

In [20]:
nlp_nl = spacy.load("nl_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

def lemmatize_text(text, lang="nl"):
    if lang == "nl":
        nlp = nlp_nl
    elif lang == "en":
        nlp = nlp_en
    else:
        raise ValueError(f"Unsupported language: {lang}")

    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]    

    return " ".join(lemmatized_tokens)

In [52]:
print(lemmatize_text("het is een zin"))

het zijn een zin


In [34]:
whapp["lemmatized"] = whapp["stopw"].apply(lambda x: lemmatize_text(x))

In [35]:
whapp["lemmatized"]

47935    dag probleem bestelling melden ontvang vandaag...
47936    goed heer mevrouw inmiddels maand ver steeds r...
47937    staan voedingswaarden vermelden fles wijn verk...
47938    bestelling [ order_number ] servicecode invull...
47939    hi bestelling net ontvangen fles inleveren gro...
                               ...                        
95906    welkom jumbo druk verzend gesprek starten endu...
95907    zoekfunctie doen app zowel mobiel tablet beide...
95908    vragen laat komtgoedenavinden lang wachten bes...
95909    goed kunnen vertellen status klacht alvast bed...
95910    hoi bestelling komen elk moment echter doen be...
Name: lemmatized, Length: 47856, dtype: object

In [36]:
# apply the function to lemmatize the text
# whapp["lemmatized"] = whapp["stopw"].apply(lemmatize_text)

In [37]:
whapp.dtypes

descr          object
source         object
clean_descr    object
stopw_descr    object
stopw          string
lemmatized     object
dtype: object

In [29]:
whapp["stopw"] = whapp["stopw_descr"]

In [30]:
whapp["stopw"] = whapp["stopw"].astype("string")

In [38]:
whapp.isnull().sum()

descr          0
source         0
clean_descr    0
stopw_descr    0
stopw          0
lemmatized     0
dtype: int64

In [25]:
# drop the nan values
whapp.dropna(subset=["stopw_descr"], inplace=True)

In [37]:
whapp["stopw_descr"][95908]

'vraagt laat komtgoedenavond lang wachten bestellen enduseroptedin berichten chat gesprekken zullen bewaard zodat jumbo beantwoorden rosanna chat grace zie bestelling gister bezorgd excuses late reactie wens fijne avond'

In [36]:
whapp["lemmatized"][95908]

'vraagt laat komtgoedenavond lang wachten bestellen enduseroptedin berichten chat gesprekken zullen bewaard zodat jumbo beantwoorden rosanna chat grace zie bestelling gister bezorgd excuse late reactie wen fijne avond'

In [40]:
whapp["stopw_descr"][47935]

'dag probleem bestelling gemeld ontvang vandaag mail akkoord terugbetaling bestellen datum melding denk goed gaat antoinette chat rubin melding gemaakt bestelnummer totale bedrag terug storten screenshot meekijken hoor graag bestelling waarover bericht [ORDER_NUMBER] bedankt foto volgens systeem zien fout gemaakt datumhet gaat hetzelfde bedrag hoop voldoende geinformeerd fijne avond'

In [39]:
whapp["lemmatized"][47935]

'dag probleem bestelling melden ontvang vandaag mail akkoord terugbetaling bestellen datum melding denken goed gaan antoinett chat rubin melding maken bestelnummer totaal bedrag terug storten screenshot meekijk hoor graag bestelling waarover bericht [ order_number ] bedanken foto volgens systeem zien fout maken datumhet gaan hetzelfde bedrag hoop voldoende informeren fijn avond'

In [42]:
# fix the masks after lemmatizing e.g. [ oder_number ] back to -> [ORDER_NUMBER]
def replace_masks(text):
    text = re.sub(r"\[\s*order\_number+\s*\]", "[ORDER_NUMBER]", text)
    text = re.sub(r"\[\s*card\_number+\s*\]", "[CARD_NUMBER]", text)
    text = re.sub(r"\[\s*phone\s*\]", "[PHONE]", text)
    text = re.sub(r"\[\s*email\s*\]", "[EMAIL]", text)
    text = re.sub(r"\[\s*promo\s*\]", "[PROMO]", text)
    text = re.sub(r"\[\s*receipt\_number+\s*\]", "[RECEIPT_NUMBER]", text)
    
    return text

In [43]:
whapp["lemmatized"] = whapp["lemmatized"].apply(replace_masks)

In [44]:
whapp["lemmatized"]

47935    dag probleem bestelling melden ontvang vandaag...
47936    goed heer mevrouw inmiddels maand ver steeds r...
47937    staan voedingswaarden vermelden fles wijn verk...
47938    bestelling [ORDER_NUMBER] servicecode invullen...
47939    hi bestelling net ontvangen fles inleveren gro...
                               ...                        
95906    welkom jumbo druk verzend gesprek starten endu...
95907    zoekfunctie doen app zowel mobiel tablet beide...
95908    vragen laat komtgoedenavinden lang wachten bes...
95909    goed kunnen vertellen status klacht alvast bed...
95910    hoi bestelling komen elk moment echter doen be...
Name: lemmatized, Length: 47856, dtype: object